In [ ]:
import numpy as np
import pandas as pd
import warnings
import joblib
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import optuna

RANDOM_SEED = 0

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
np.random.seed(RANDOM_SEED)

# =============================================================================
# CONFIGURATION
# =============================================================================
print("="*80)
print("GRADIENT BOOSTING HYPERPARAMETER OPTIMIZATION")
print("="*80)

# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../../data/preprocessed/X_train.pkl')
y_train = joblib.load('../../data/preprocessed/y_train.pkl')
X_test = joblib.load('../../data/preprocessed/X_test.pkl')
metadata = joblib.load('../../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Target shape: {y_train.shape}")
print(f"✓ Test shape: {X_test.shape}")

# =============================================================================
# OPTIMIZATION SETTINGS
# =============================================================================
FAST_MODE = False  # Set to False for more thorough search
N_TRIALS = 15 if FAST_MODE else 200
N_FOLDS = 5
N_JOBS_OPTUNA = 1  # Parallel trials

print(f"\n{'='*60}")
print(f"OPTIMIZATION SETTINGS")
print(f"{'='*60}")
print(f"Mode: {'FAST' if FAST_MODE else 'THOROUGH'}")
print(f"Trials: {N_TRIALS}")
print(f"Folds: {N_FOLDS}")
print(f"Parallel jobs: {N_JOBS_OPTUNA}")

# =============================================================================
# OBJECTIVE FUNCTION
# =============================================================================
def objective_gbr(trial, X, y):
    """Objective function for GradientBoosting optimization"""

    start_time = time.time()

    print(f"\n{'='*60}")
    print(f"Trial {trial.number}")
    print(f"{'='*60}")
    print(f"Parametri testati: {params}")
    
    # Hyperparameters to optimize
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'random_state': RANDOM_SEED,
    }

    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        
        # Handle both pandas Series and numpy arrays
        if isinstance(y, pd.Series):
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        else:
            y_tr, y_val = y[train_idx], y[val_idx]

        model = GradientBoostingRegressor(**params)
        model.fit(X_tr, y_tr)

        preds = np.clip(model.predict(X_val), 0, 100)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(rmse)
        
        print(f"  Fold {fold_idx + 1}: RMSE = {rmse:.4f}")
    
    optimization_time = time.time() - start_time
    mean_score = np.mean(scores)
    print(f"RMSE medio: {mean_score:.4f}")
    print(f"Elapsed Time: {optimization_time:.1f}s")
    print(f"{'='*60}\n")
    
    return mean_score

# =============================================================================
# RUN OPTIMIZATION
# =============================================================================
print("\n" + "="*80)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*80)

start_time = time.time()

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
)

study.optimize(
    lambda trial: objective_gbr(trial, X_train, y_train),
    n_trials=N_TRIALS,
    show_progress_bar=True,
    #n_jobs=N_JOBS_OPTUNA
)

optimization_time = time.time() - start_time

# =============================================================================
# RESULTS
# =============================================================================
print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(f"Best RMSE: {study.best_value:.6f}")
print(f"Optimization time: {optimization_time:.1f}s")
print(f"\nBest parameters:")
for param, value in sorted(study.best_params.items()):
    print(f"  {param:20s}: {value}")

# =============================================================================
# SAVE RESULTS
# =============================================================================
# Save best parameters
best_params = study.best_params.copy()
best_params['random_state'] = RANDOM_SEED

joblib.dump(best_params, 'gradientboosting_params.pkl')
print("\n✓ Parameters saved to: gradientboosting_params.pkl")

# Save optimization history
history = pd.DataFrame({
    'trial': [t.number for t in study.trials],
    'value': [t.value for t in study.trials],
    'params': [str(t.params) for t in study.trials]
})
history.to_csv('gradientboosting_history.csv', index=False)
print("✓ History saved to: gradientboosting_history.csv")

# Save study object
joblib.dump(study, 'gradientboosting_study.pkl')
print("✓ Study saved to: gradientboosting_study.pkl")

# Save summary
summary = {
    'model': 'GradientBoosting',
    'best_rmse': study.best_value,
    'n_trials': N_TRIALS,
    'n_folds': N_FOLDS,
    'optimization_time': optimization_time,
    'best_params': best_params
}
joblib.dump(summary, 'gradientboosting_summary.pkl')
print("✓ Summary saved to: gradientboosting_summary.pkl")

# =============================================================================
# TRAIN FINAL MODEL ON FULL TRAINING DATA
# =============================================================================
print("\n" + "="*80)
print("TRAINING FINAL MODEL ON FULL DATASET")
print("="*80)

final_model = GradientBoostingRegressor(**best_params)
final_model.set_params(verbose=1)  # Show training progress

print("Training final model...")
final_model.fit(X_train, y_train)

# Save final model
joblib.dump(final_model, 'gradientboosting_final_model.pkl')
print("\n✓ Final model saved to: gradientboosting_final_model.pkl")

# =============================================================================
# GENERATE PREDICTIONS AND SUBMISSION FILE
# =============================================================================
print("\n" + "="*80)
print("GENERATING PREDICTIONS")
print("="*80)

# Make predictions on test set
test_predictions = final_model.predict(X_test)

# Clip predictions to valid range [0, 100]
test_predictions = np.clip(test_predictions, 0, 100)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': range(len(X_train), len(test_predictions) + len(X_train)),
    'test_score': test_predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)
print("✓ Submission file saved to: submission.csv")

print(f"\nSubmission statistics:")
print(f"  Min prediction: {test_predictions.min():.2f}")
print(f"  Max prediction: {test_predictions.max():.2f}")
print(f"  Mean prediction: {test_predictions.mean():.2f}")
print(f"  Median prediction: {np.median(test_predictions):.2f}")
print(f"  Std prediction: {test_predictions.std():.2f}")
print(f"  Shape: {submission.shape}")

# =============================================================================
# FEATURE IMPORTANCE
# =============================================================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE")
print("="*80)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 most important features:")
print(feature_importance.head(10).to_string(index=False))

# Save feature importance
feature_importance.to_csv('gradientboosting_feature_importance.csv', index=False)
print("\n✓ Feature importance saved to: gradientboosting_feature_importance.csv")

# =============================================================================
# TRAINING PROGRESS ANALYSIS
# =============================================================================
print("\n" + "="*80)
print("TRAINING PROGRESS ANALYSIS")
print("="*80)

# Get training scores (on training data)
train_score = final_model.train_score_

print(f"Initial training score (first iteration): {train_score[0]:.6f}")
print(f"Final training score: {train_score[-1]:.6f}")
print(f"Total improvement: {train_score[0] - train_score[-1]:.6f}")

# Plot training progress
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training score over iterations
ax1 = axes[0]
ax1.plot(range(1, len(train_score) + 1), train_score, linewidth=2)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Training Score (Deviance)')
ax1.set_title('Training Progress')
ax1.grid(True, alpha=0.3)

# Feature importance (top 15)
ax2 = axes[1]
top_features = feature_importance.head(15)
y_pos = np.arange(len(top_features))
ax2.barh(y_pos, top_features['importance'], color='skyblue', edgecolor='black')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(top_features['feature'], fontsize=9)
ax2.invert_yaxis()
ax2.set_xlabel('Importance')
ax2.set_title('Top 15 Feature Importances')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('gradientboosting_analysis.png', dpi=300, bbox_inches='tight')
print("\n✓ Training analysis plot saved to: gradientboosting_analysis.png")
plt.show()

# =============================================================================
# MODEL STATISTICS
# =============================================================================
print("\n" + "="*80)
print("MODEL STATISTICS")
print("="*80)
print(f"Number of estimators: {best_params['n_estimators']}")
print(f"Learning rate: {best_params['learning_rate']:.4f}")
print(f"Max depth: {best_params['max_depth']}")
print(f"Subsample ratio: {best_params['subsample']:.2f}")
print(f"Total number of leaves across all trees: {sum(est[0].tree_.n_leaves for est in final_model.estimators_)}")
print(f"Average leaves per tree: {sum(est[0].tree_.n_leaves for est in final_model.estimators_) / len(final_model.estimators_):.1f}")

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "="*80)
print("GRADIENT BOOSTING OPTIMIZATION COMPLETE!")
print("="*80)
print(f"✓ Best CV RMSE: {study.best_value:.6f}")
print(f"✓ Final model trained on {len(X_train)} samples")
print(f"✓ Model has {best_params['n_estimators']} estimators")
print(f"✓ Predictions generated for {len(X_test)} test samples")
print(f"✓ All results saved")
print("\nFiles created:")
print("  • gradientboosting_params.pkl")
print("  • gradientboosting_history.csv")
print("  • gradientboosting_study.pkl")
print("  • gradientboosting_summary.pkl")
print("  • gradientboosting_final_model.pkl")
print("  • gradientboosting_feature_importance.csv")
print("  • gradientboosting_analysis.png")
print("  • submission.csv")
print("="*80)